# Titre

In [ ]:
import torch
import random
import matplotlib.pyplot as plt
from accelerate.test_utils.testing import get_backend
from transformers import AutoTokenizer, AutoModelForCausalLM

## Code setup

### Model loading

In [ ]:
# Model list (all tested)
# gpt2
# gpt2-large
# EleutherAI/gpt-j-6B
# EleutherAI/gpt-neox-20b

MODEL_NAME = "gpt2-large" 

device,_,_ = get_backend()

# Load model
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)

# Load tokenizer 
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Set to evaluation mode
model.eval()

### Useful functions

In [ ]:
def generate_text(prompt: str, max_length: int) -> str:
    """
    Output AI-generated text using the chosen model

    Args:
        prompt (str): prompt to generate text
        max_length (int): the maximum length (~ number of words) of the generated text

    Returns:
        str: generated text
    """
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        output = model.generate(**inputs, max_length=max_length, do_sample=True, temperature=0.7)
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
def compute_log_prob(text: str) -> float:
    """
    Compute the log prob of a given text under the chosen model

    Args:
        text (str): input text for which to compute the log prob

    Returns:
        float: input text log prob
    """
    tokens = tokenizer(text, return_tensors="pt")
    input_ids = tokens["input_ids"]
    n_tokens = input_ids.shape[1]
    
    with torch.no_grad():
        outputs = model(**tokens, labels=input_ids)
        # negative of the NLL per token = log prob
        log_prob = -outputs.loss.item() * n_tokens # total NLL
    return log_prob

In [ ]:
def compute_perplexity(text: str) -> float:
    """
    Compute the perplexity score of a given text using the chosen model

    Args:
        text (str): input text for which to compute perplexity

    Returns:
        float: text perplexity score 
    """
    tokens = tokenizer(text, return_tensors="pt")
    input_ids = tokens["input_ids"]

    with torch.no_grad():
        outputs = model(**tokens, labels=input_ids)
        log_prob = outputs.loss # NLL per token
        
    perplexity = torch.exp(log_prob) if log_prob < 100 else float("inf") # overflow possible

    return perplexity

In [ ]:
def dummy_perturbation(text: str) -> str:
    """
    Randomly swaps two adjacent words to create a perturbed version of the text

    Args:
        text (str): the input text to be perturbed

    Returns:
        str: the perturbed text 
    """
    words = text.split()
    if len(words) > 3:
        i = random.randint(0, len(words)-2)
        words[i], words[i + 1] = words[i + 1], words[i]
    return " ".join(words)

In [ ]:
# TODO implement the T5 perturbation of the paper

def perturbation(text: str) -> str:
    """
    T5 perturbation

    Args:
        text (str): the input text to be perturbed

    Returns:
        str: the perturbed text 
    """
    return None

In [ ]:
def detectgpt_score(text: str, num_perturbations: int) -> bool:
    """
    Implement DetectGPT algorithm 1

    Args:
        text (str): input text to be analyzed
        num_perturbations (int): number of perturbed versions of the text to generate

    Returns:
        bool: true if the text is likely model-generated
    """
    original_log_prob = compute_log_prob(text) # log prob of the original text

    # Generate perturbed texts + compute their log prob
    perturbed_texts = [dummy_perturbation(text) for _ in range(num_perturbations)]
    perturbed_log_probs = [compute_log_prob(pt) for pt in perturbed_texts]

    mu = sum(perturbed_log_probs) / num_perturbations # mean log probability of the perturbed texts

    d = original_log_prob - mu # estimate perturbation discrepancy d

    variance = sum((log_prob - mu) ** 2 for log_prob in perturbed_log_probs) / (num_perturbations - 1) # variance of the log probabilities
    sigma = variance ** 0.5  # standard deviation

    score = d / sigma if sigma > 0 else 0 

    return score

## Experiments

### 1. Simple use

In [ ]:
# Example usage

max_length = 60
prompt = "In a faraway galaxy, where no humans exist"


ai_text = generate_text(prompt,max_length)

# Human text from CNN
human_text = "But Bhaduri found it increasingly hard to secure work after more women began partaking in jatra productions in the 1960s and 1970s. By the time he met Kishore, who was running a theater publication at the time, the actor was in his 60s and only performing a handful of times a year for the equivalent of $1 a night."

print("AI text:", ai_text)
print("human text:", human_text)

In [ ]:
num_perturbations = 10
ai_text_score = detectgpt_score(ai_text,num_perturbations)
human_text_score = detectgpt_score(human_text,num_perturbations)

print("AI text DetectGPT score:", ai_text_score)
print("Human text DetectGPT score:", human_text_score)

### 2. Perplexity score

In [ ]:
# Compute perplexity for AI-generated text and human text

max_length = 60
prompt = "In a faraway galaxy, where no humans exist"
ai_text = generate_text(prompt, max_length)

human_text = "But Bhaduri found it increasingly hard to secure work after more women began partaking in jatra productions in the 1960s and 1970s. By the time he met Kishore, who was running a theater publication at the time, the actor was in his 60s and only performing a handful of times a year for the equivalent of $1 a night."

# Compute perplexity for AI-generated text
perplexity_ai = compute_perplexity(ai_text)
perplexity_ai_perturbed = compute_perplexity(dummy_perturbation(ai_text))

# Compute perplexity for human-written text
perplexity_human = compute_perplexity(human_text)
perplexity_human_perturbed = compute_perplexity(dummy_perturbation(human_text))

print(f"Perplexity AI: {perplexity_ai}")
print(f"Perplexity AI (perturbed): {perplexity_ai_perturbed}")
print(f"Perplexity human: {perplexity_human}")
print(f"Perplexity human (perturbed): {perplexity_human_perturbed}")

In [ ]:
# Scatter plot analysis
num_samples = 100
max_length = 60
prompts = ["In a faraway galaxy, where no humans exist" for _ in range(num_samples)]

# Generate AI-generated texts based on the prompts
ai_texts = [generate_text(prompt, max_length) for prompt in prompts]

# Compute perplexity before and after perturbation
perplexities_before = [compute_perplexity(text) for text in ai_texts]
perplexities_after = [compute_perplexity(dummy_perturbation(text)) for text in ai_texts]

In [ ]:
# Create scatter plot
plt.scatter(perplexities_before, perplexities_after,marker='x',c='r')

plt.xlabel("Perplexity before perturbation")
plt.ylabel("Perplexity after perturbation")
plt.plot([0,30],[0,30],'b--')
plt.title(f"Perplexity before vs after perturbation for {num_samples} AI-generated texts")
plt.grid(True)
plt.show()